In [59]:
from PIL import Image, ImageDraw
import dlib
import imageio.v2 as imageio
import os
import albumentations as A
import numpy as np
import concurrent.futures
import random
import shutil

In [7]:
def load_png(image_path):
    try:
        # Membaca gambar menggunakan imageio
        img_array = imageio.imread(image_path)
        
        # Mengembalikan array NumPy
        return img_array

    except Exception as e:
        print("Terjadi kesalahan:", e)
        return None

In [8]:
def crop_face_png(image_path, output_folder):
    # Memuat gambar
    img_array = load_png(image_path)

    if img_array is not None:
        # Menginisialisasi detector wajah
        face_detector = dlib.get_frontal_face_detector()

        # Mendeteksi wajah pada gambar
        faces = face_detector(img_array)

        # Menginisialisasi prediktor landmark wajah
        landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

        for face in faces:
            # Mendapatkan landmark wajah
            landmarks = landmark_predictor(img_array, face)

            # Mendapatkan koordinat landmark untuk bagian wajah yang akan di-crop
            left_eye_x = landmarks.part(36).x
            left_eye_y = landmarks.part(37).y
            right_eye_x = landmarks.part(39).x
            right_eye_y = landmarks.part(41).y
            left_eyebrow_x = landmarks.part(17).x  # Koordinat landmark alis kiri
            left_eyebrow_y = landmarks.part(19).y
            right_eyebrow_x = landmarks.part(26).x  # Koordinat landmark alis kanan
            right_eyebrow_y = landmarks.part(24).y
            nose_x = landmarks.part(30).x
            nose_y = landmarks.part(30).y
            ear_left_x = landmarks.part(0).x
            ear_left_y = landmarks.part(0).y
            ear_right_x = landmarks.part(16).x
            ear_right_y = landmarks.part(16).y

            # Menentukan batas crop untuk wajah
            top_margin = int((left_eyebrow_y) * 0.1)  # 20% dari tinggi alis sebagai margin atas

            # komen jika ingin dapatkan width dari gambar
            left_bound = left_eyebrow_x
            right_bound = right_eyebrow_x

            top_bound = left_eyebrow_y - top_margin
            bottom_bound = ear_left_y + (nose_y - ear_left_y)  # Menggunakan setengah hidung sebagai batas bawah

            # Mendapatkan batas gambar asli
            original_width, original_height = img_array.shape[1], img_array.shape[0]

            # Menentukan batas crop untuk wajah
            #left_bound = 0  # Batas kiri tidak boleh kurang dari 0
            #right_bound = original_width  # Batas kanan tidak boleh lebih dari lebar gambar

            # # Crop bagian wajah dari gambar
            face_image = Image.fromarray(img_array[top_bound:bottom_bound, left_bound:right_bound])

            # not croping
            # face_image = Image.fromarray(img_array)

            # Menampilkan gambar wajah yang di-crop (opsional)
            # face_image.show()

            # Simpan gambar wajah yang di-crop ke folder yang sesuai
            output_path = os.path.join(output_folder, os.path.basename(image_path))
            face_image.save(output_path)

In [13]:
# Ganti path file dengan path gambar PNG yang ingin Anda muat
#image_path = "/Users/bayu/Library/CloudStorage/OneDrive-PTCitilinkIndonesia/kuliah/tesis/image/CS229/CK+/happiness/S050_006_00000023.png"
#rootFold = "/Volumes/DiskFile/Kuliah/bahan thesis/ImageCrop/allwidth"

# Melakukan crop pada bagian wajah dari gambar
#crop_face_png(image_path, rootFold)

In [18]:
# Fungsi untuk menerapkan augmentasi pada gambar
def apply_augmentation(image_path, output_image_path, output_dir):

    # Load gambar
    image = np.array(Image.open(image_path))
    
    # Tentukan augmentasi yang ingin dilakukan
    transform= A.Compose([
            A.HorizontalFlip(p=.5),
            A.Rotate(limit=5, p=.25),
            A.RandomBrightnessContrast(p=.5),
            A.RandomGamma(p=.5)
        ]) 
    
    # Terapkan augmentasi pada gambar
    augmented_image = transform(image=image)['image']
    
    # Simpan gambar yang sudah diubah
    #image_name = os.path.basename(image_path)
    #output_path = os.path.join(output_dir, image_name)
    Image.fromarray(augmented_image).save(output_image_path)

In [65]:
# Fungsi untuk menggandakan jumlah gambar dalam satu folder
def duplicate_images_in_folder(folder_path, output_dir):
    # Pastikan output_dir ada atau buat jika belum ada
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Hitung jumlah gambar maksimum dalam satu folder
    max_images = max(len(os.listdir(folder_path)), 1)  # Jika tidak ada gambar, setidaknya akan ada satu gambar yang dibuat
    print('length max_images')
    print(max_images)

    print('length folder_path')
    print(len(os.listdir(folder_path)))
    print(' ')

    # Hitung jumlah gambar yang akan dihasilkan untuk gambar ini
    # num_augmented_images = max_images * 2
    num_augmented_images = 500
    delta_image = num_augmented_images-max_images

    # Membuat daftar semua file dalam folder
    file_list = [file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file))]
    # Memilih secara acak satu file dari daftar file
    random_file = random.choice(file_list)
    # Path lengkap ke file yang dipilih secara acak
    random_file_path = os.path.join(folder_path, random_file)
    print('random_file_path '+random_file_path)

    for i in range(delta_image):
        # Loop melalui setiap gambar dalam folder
        # for filename in os.listdir(folder_path):
        #     if filename.lower().endswith('.jpg') or filename.lower().endswith('.png'):
        #         image_path = os.path.join(folder_path, filename)
                
        #         # Simpan gambar hasil augmentasi dengan menambahkan nomor iterasi ke nama file
        #         output_image_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_{i}.jpg")
        #         apply_augmentation(image_path, output_image_path, output_dir)
        
        # Simpan gambar hasil augmentasi dengan menambahkan nomor iterasi ke nama file
        output_image_path = os.path.join(output_dir, f"{os.path.splitext(random_file)[0]}_{i}.jpg")
        apply_augmentation(random_file_path, output_image_path, output_dir)
    
    

In [66]:
def balance_image(output_folder):
    print('output_folder+ ',output_folder)
    print('length output_folder')
    print(len(os.listdir(output_folder)))
    print(' ')


        
    # Loop melalui setiap folder dalam root_folder
    for folder_name in os.listdir(output_folder):
        print(f'folder_name {folder_name}')
        folder_path = os.path.join(output_folder, folder_name)
        print(f'folder_path {folder_path}')
        # Pastikan folder_path adalah direktori
        if os.path.isdir(folder_path):
            #output_dir = os.path.join(output_folder, folder_name + '_augmented')
            output_dir = os.path.join(output_folder+'_augmented', folder_name)
            duplicate_images_in_folder(folder_path, output_dir)

            for filename in os.listdir(folder_path):
                src_file = os.path.join(folder_path, filename)
                # Menyalin file dari sumber ke tujuan
                shutil.copy(src_file, output_dir)
            
    

In [25]:
# Fungsi untuk melakukan iterasi melalui semua folder dan gambar dalamnya
def process_all_images(root_folder, root_folder_result):
    for foldername, subfolders, filenames in os.walk(root_folder):
        for filename in filenames:
            # Periksa apakah file memiliki ekstensi gambar yang diizinkan
            if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
                # Buat path lengkap ke gambar
                image_path = os.path.join(foldername, filename)

                # Buat path folder output yang sesuai dengan folder asal
                output_folder = os.path.join(root_folder_result, os.path.relpath(foldername, root_folder))

                # Pastikan folder output sudah ada, jika belum, buat folder baru
                os.makedirs(output_folder, exist_ok=True)

                # Lakukan cropping pada gambar dan simpan hasil ke folder yang sesuai
                crop_face_png(image_path, output_folder)

    
                

In [67]:
# Ganti path folder utama yang berisi semua subfolder dan gambar
root_folder = "/Users/bayu/Library/CloudStorage/OneDrive-PTCitilinkIndonesia/kuliah/tesis/image/CS229/TFEID_class"
root_folder_result = "/Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio"

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     # Panggil fungsi untuk memproses semua gambar dalam folder
#     print('proses crop image')
#     executor.submit(process_all_images(root_folder, root_folder_result))

print('done process image')
print('start balance')
# Lakukan balancing pada gambar
balance_image(root_folder_result)
print('done balance')


done process image
start balance
output_folder+  /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio
length output_folder
9
 
folder_name .DS_Store
folder_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/.DS_Store
folder_name contempt
folder_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/contempt
length max_images
68
length folder_path
68
 
random_file_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/contempt/f11_dfh_clx.jpg
folder_name fear
folder_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/fear
length max_images
40
length folder_path
40
 
random_file_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/fear/f18_dfh_fx.jpg
folder_name surprise
folder_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/surprise
length max_images
36
length folder_path
36
 
random_file_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/surprise/f05_dfh_px.JPG
folder_name sadness
folder_path /Volumes/DiskFile/Kuliah/tesis_code/ImageCrop/perio/sadness
l